# Named Volumes 

In [113]:
cd notebooks/docker-code/docker-volumes-deep/

[Errno 2] No such file or directory: 'notebooks/docker-code/docker-volumes-deep/'
/notebooks/docker-code/docker-volumes-deep


In [2]:
ls

Dockerfile  feedback/  package.json  pages/  public/  server.js  temp/


In [5]:
!docker image ls

REPOSITORY                 TAG                 IMAGE ID            CREATED             SIZE
docker-volumes-0           latest              c14c5c36396d        9 minutes ago       916MB
<none>                     <none>              92d76da46277        11 minutes ago      261MB
ansible-101_bastion        latest              365bea7882ac        11 minutes ago      261MB
ansible-101_web            latest              365bea7882ac        11 minutes ago      261MB
debian                     bullseye            7df9c1a77ca2        39 hours ago        124MB
node                       14                  1d12470fa662        2 months ago        912MB
ioggstream/python-course   latest              f2bdf3f866fd        5 years ago         1.65GB


### Buildiamo la nuova immagine

In [6]:
!docker build . -t docker-volumes-0:named

Sending build context to Docker daemon  13.82kB
Step 1/8 : FROM node:14
 ---> 1d12470fa662
Step 2/8 : WORKDIR /app
 ---> Using cache
 ---> 9a5a10b8cedf
Step 3/8 : COPY package.json .
 ---> 3441282a1a99
Step 4/8 : RUN npm install
 ---> Running in 6582686ab3a2

> nodemon@2.0.4 postinstall /app/node_modules/nodemon
> node bin/postinstall || exit 0

Love nodemon? You can now support the project via the open collective:
 > https://opencollective.com/nodemon/donate

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN optional SKIPPING OPTIONAL DEPENDENCY: fsevents@~2.3.2 (node_modules/chokidar/node_modules/fsevents):
npm WARN notsup SKIPPING OPTIONAL DEPENDENCY: Unsupported platform for fsevents@2.3.2: wanted {"os":"darwin","arch":"any"} (current: {"os":"linux","arch":"x64"})
npm WARN data-volume-example@1.0.0 No description
npm WARN data-volume-example@1.0.0 No repository field.

added 172 packages from 88 contributors and audited 173 packages in 5.755s

In [8]:
# Tralasciando le rige npm, cosa noti di particolare negli step di build?

### Creazione del nuovo container con un Named Volume

In [11]:
!docker image ls

REPOSITORY                 TAG                 IMAGE ID            CREATED             SIZE
docker-volumes-0           named               79d2850f99db        3 minutes ago       922MB
docker-volumes-0           latest              c14c5c36396d        13 minutes ago      916MB
ansible-101_bastion        latest              365bea7882ac        15 minutes ago      261MB
ansible-101_web            latest              365bea7882ac        15 minutes ago      261MB
<none>                     <none>              92d76da46277        15 minutes ago      261MB
debian                     bullseye            7df9c1a77ca2        39 hours ago        124MB
node                       14                  1d12470fa662        2 months ago        912MB
ioggstream/python-course   latest              f2bdf3f866fd        5 years ago         1.65GB


In [15]:
#Dovresti già conoscere il comando usato per eseguire il nuovo container, prova ad indovinare
!docker run --name volumes-deep -p 8080:80 -v named:/app/feedback --rm docker-volumes-0:named

docker: Error response from daemon: driver failed programming external connectivity on endpoint volumes-deep (fce1262bdc492ef1972979254da48a960832c94afb4fd35263eee41b4942c176): Bind for 0.0.0.0:8080 failed: port is already allocated.


In [18]:
#Come risolvere il problema?
!docker ps
!docker stop f0088d8dc37e
!docker rm f0088d8dc37e # dovrebbe già essere cancellato perchè lanciato con --rm

CONTAINER ID        IMAGE                      COMMAND                  CREATED             STATUS              PORTS                    NAMES
f0088d8dc37e        docker-volumes-0           "docker-entrypoint.s…"   12 minutes ago      Up 12 minutes       0.0.0.0:8080->80/tcp     volumes-test-0
c8aa1cf60e9c        ansible-101_bastion        "/usr/sbin/sshd -D -e"   18 minutes ago      Up 18 minutes                                ansible-101_bastion_1
c87d2dcc853c        ansible-101_web            "/usr/sbin/sshd -D -e"   18 minutes ago      Up 18 minutes                                ansible-101_web_2
6ff45d1c03e5        92d76da46277               "/usr/sbin/sshd -D -e"   18 minutes ago      Up 18 minutes                                ansible-101_web_1
758b6da0bbf9        ioggstream/python-course   "/bin/sh -c '/usr/lo…"   19 minutes ago      Up 19 minutes       0.0.0.0:8888->8888/tcp   ansible-101_course_1
f0088d8dc37e
Error: No such container: f0088d8dc37e


In [19]:
!docker run --name volumes-deep -p 8080:80 -v named:/app/feedback --rm -d docker-volumes-0:named


> data-volume-example@1.0.0 start /app
> nodemon server.js

[nodemon] 2.0.4
[nodemon] to restart at any time, enter `rs`
[nodemon] watching path(s): *.*
[nodemon] watching extensions: js,mjs,json
[nodemon] starting `node server.js`
TEST!!!!!
TEST!!!!!
^C


### Verifica presenza del volume

In [20]:
!docker ps 

CONTAINER ID        IMAGE                      COMMAND                  CREATED              STATUS              PORTS                    NAMES
f21ca64f902c        docker-volumes-0:named     "docker-entrypoint.s…"   About a minute ago   Up About a minute   0.0.0.0:8080->80/tcp     volumes-deep
c8aa1cf60e9c        ansible-101_bastion        "/usr/sbin/sshd -D -e"   26 minutes ago       Up 26 minutes                                ansible-101_bastion_1
c87d2dcc853c        ansible-101_web            "/usr/sbin/sshd -D -e"   26 minutes ago       Up 26 minutes                                ansible-101_web_2
6ff45d1c03e5        92d76da46277               "/usr/sbin/sshd -D -e"   26 minutes ago       Up 26 minutes                                ansible-101_web_1
758b6da0bbf9        ioggstream/python-course   "/bin/sh -c '/usr/lo…"   26 minutes ago       Up 26 minutes       0.0.0.0:8888->8888/tcp   ansible-101_course_1


In [22]:
!docker inspect f21ca64f902c | grep volume

        "Name": "/volumes-deep",
                "Type": "volume",
                "Source": "/var/lib/docker/volumes/0392ffd3462106d0a5772a3bda21e31420eae1c0849cd5a671de53f06948965b/_data",
                "Type": "volume",
                "Source": "/var/lib/docker/volumes/named/_data",
            "Image": "docker-volumes-0:named",


In [114]:
!docker volume ls

DRIVER              VOLUME NAME
local               372818fb96642f6be96ff2b3ced017a5185102c12cf3889d02ba879d4c050ce0
local               feedback
local               named


In [24]:
!docker stop f21ca64f902c

f21ca64f902c


In [25]:
!docker rm f21ca64f902c # se lanciato senza --rm

Error: No such container: f21ca64f902c


### Riavvio container

In [30]:
!docker run --name volumes-deep -p 8080:80 -v named:/app/feedback --rm -d docker-volumes-0:named

1c8c092bfb64e19a5487a6937f03a72d9d9ad34d92f7a94b7fcab82f5172156f


In [28]:
# Apri il browser
# Verifica presenza del feedback precedentemente lasciato

In [54]:
!docker stop 1c8c092bfb64e

Error response from daemon: No such container: 1c8c092bfb64e


# Anonymous Volumes

In [108]:
!docker volume ls

DRIVER              VOLUME NAME
local               feedback
local               named


In [103]:
!docker volume inspect feedback

[
    {
        "CreatedAt": "2023-06-14T13:49:21Z",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/feedback/_data",
        "Name": "feedback",
        "Options": null,
        "Scope": "local"
    }
]


In [104]:
!docker volume inspect named

[
    {
        "CreatedAt": "2023-06-14T14:08:32Z",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/named/_data",
        "Name": "named",
        "Options": null,
        "Scope": "local"
    }
]


### Creiamo un container senza volumi

In [109]:
!docker run --name volumes-deep-2 -p 8081:80 --rm -d docker-volumes-0:named

d5cd57762a48645e50a9aee82680d1fcdd57caeeea00ba4437776165faaa6614


In [110]:
!docker ps

CONTAINER ID        IMAGE                      COMMAND                  CREATED             STATUS              PORTS                    NAMES
d5cd57762a48        docker-volumes-0:named     "docker-entrypoint.s…"   7 seconds ago       Up 7 seconds        0.0.0.0:8081->80/tcp     volumes-deep-2
c8aa1cf60e9c        ansible-101_bastion        "/usr/sbin/sshd -D -e"   4 hours ago         Up 4 hours                                   ansible-101_bastion_1
c87d2dcc853c        ansible-101_web            "/usr/sbin/sshd -D -e"   4 hours ago         Up 4 hours                                   ansible-101_web_2
6ff45d1c03e5        92d76da46277               "/usr/sbin/sshd -D -e"   4 hours ago         Up 4 hours                                   ansible-101_web_1
758b6da0bbf9        ioggstream/python-course   "/bin/sh -c '/usr/lo…"   4 hours ago         Up 4 hours          0.0.0.0:8888->8888/tcp   ansible-101_course_1


In [117]:
!docker volume ls

DRIVER              VOLUME NAME
local               372818fb96642f6be96ff2b3ced017a5185102c12cf3889d02ba879d4c050ce0
local               feedback
local               named


In [118]:
#In realtà un nuovo volume è già stato creato
#Questo perchè all'interno del Dockerfile è stato inserito il comando VOLUME

In [119]:
!cat Dockerfile

FROM node:14

WORKDIR /app

COPY package.json .

RUN npm install

COPY . .

EXPOSE 80

# VOLUME [ "/app/node_modules" ]

VOLUME ["/temp"]

CMD [ "npm", "start" ]

[Modifichiamo il Dockerfile](/tree/notebooks/docker-code/docker-volumes-deep)

### Buildiamo la nuova immagine

In [124]:
!docker build . -t docker-volumes-0:anonymous

Sending build context to Docker daemon  13.82kB
Step 1/7 : FROM node:14
 ---> 1d12470fa662
Step 2/7 : WORKDIR /app
 ---> Using cache
 ---> 9a5a10b8cedf
Step 3/7 : COPY package.json .
 ---> Using cache
 ---> 3441282a1a99
Step 4/7 : RUN npm install
 ---> Using cache
 ---> 47ab7c88ba5d
Step 5/7 : COPY . .
 ---> Using cache
 ---> bf4bf32cad8a
Step 6/7 : EXPOSE 80
 ---> Using cache
 ---> 6ca917725d8b
Step 7/7 : CMD [ "npm", "start" ]
 ---> Using cache
 ---> 36e050044ea8
Successfully built 36e050044ea8
Successfully tagged docker-volumes-0:anonymous


### Stoppiamo e cancelliamo il container creato

In [126]:
!docker stop d5cd57762a48645e50a9aee82680d1fcdd57caeeea00ba4437776165faaa6614

d5cd57762a48645e50a9aee82680d1fcdd57caeeea00ba4437776165faaa6614


### Lanciamo un nuovo comando senza volume

In [127]:
!docker run --name volumes-deep-3 -p 8081:80 --rm -d docker-volumes-0:anonymous

09110e8968b0f6fd395964d6cb60dd76edf4b43a962e18181511e68bce6d04ba


In [129]:
### Verifichiamo il numero di volumi
### Possiamo vedere che nessun nuovo volume è stato creato
!docker volume ls

DRIVER              VOLUME NAME
local               feedback
local               named


In [130]:
!docker stop 09110e8968b0f6fd395964d6cb60dd76edf4b43a962e18181511e68bce6d04ba

09110e8968b0f6fd395964d6cb60dd76edf4b43a962e18181511e68bce6d04ba


### Gli Anonymous Volumes possono essere creati anche non specificando il nome del volume dal comando Docker

In [135]:
!docker run --name volumes-deep -p 8080:80 -v /app/feedback --rm -d docker-volumes-0:anonymous

403f299df63fc9cdf8ab89c58f333702e8c8dd2496a85a9adbfe9f7bb538461f


In [137]:
#Possiamo vedere che l'anonymous volume è stato creato
!docker volume ls

DRIVER              VOLUME NAME
local               823eaf39b10bc90cb75539c8508062649f627b349f67eaa2e9cb81f2c36a83c6
local               feedback
local               named


In [138]:
!docker stop 403f299df63fc9cdf8ab89c58f333702e8c8dd2496a85a9adbfe9f7bb538461f

403f299df63fc9cdf8ab89c58f333702e8c8dd2496a85a9adbfe9f7bb538461f


In [139]:
!docker volume ls

DRIVER              VOLUME NAME
local               feedback
local               named


# Binding Mount

In [ ]:
!docker run --name volumes-deep -p 8080:80 -v "global-path":/app/feedback --rm -d docker-volumes-0:anonymous

In [140]:
#Parleremo più nel dettaglio di questi container più avanti

## Continuare con la parte di networking nel prossimo notebook